In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

!pip install backtrader
import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import seaborn as sns
import sys
import random
sys.path.append('..')
#from decisivealpha.montecarlo import MonteCarlo #got error while !pip or !apt install in Google colab 

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#!pip install decisivealpha
!apt install -y decisivealpha

# Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    # RSI parameters    
    params = (
        ('ema_long_period', 26),
        ('ema_short_period',12),
        ('printlog', False),
    )
    

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
       
        
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.ema_long = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_long_period) #kevin technical indicator chosen 
        self.ema_short = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_short_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Buy if 14 day rsi less than 20    
            if self.ema_long[0] - self.ema_short[0] > 0: # kevin to change conditions
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    

        else:
            # Sell if 7 day rsi over 80
            if self.ema_long[0] - self.ema_short[0] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(EMA long Period %2d, EMA short Period %2d) Ending Value %.2f' %
                 (self.params.ema_long_period, self.params.ema_short_period,  self.broker.getvalue()), doprint=True)

# Analyzers

In [ ]:
class RSIAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.ema_long = self.strategy.params.ema_long_period
        self.ema_short = self.strategy.params.ema_short_period
        self.equity = []
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None

    def next(self):
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()

        if trade.isclosed:
            self.num_trades += 1
            self.equity.append(self.strategy.broker.getvalue())
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        equity = np.asarray([self.init_cash,] + self.equity)
        
        results = {
            'params': (self.ema_long, self.ema_short),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'trades': np.diff(equity).tolist()
        }
        
        return results

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.rsiA.get_analysis()
        params.append(r['params'])
        n_trades.append(r['num_trades'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join(r'/content/gdrive/My Drive/DecisiveWorkflowResearch/data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

# In-sample Parameters

In [ ]:
def get_periods(start_is, is_duration, oos_duration, wf_offset,wf_cycles):
    periods=[]
    start_is=pd.Timestamp(start_is)
    start_oos=start_is+pd.DateOffset(**is_duration)
    end_oos=start_oos+pd.DateOffset(**oos_duration)
    for i in range(wf_cycles):
        periods.append({'run':i,
                        'is':(start_is, start_oos),
                        'oos':(start_oos, end_oos)})
        start_is+=pd.DateOffset(**wf_offset)
        start_oos+=pd.DateOffset(**wf_offset)
        end_oos+=pd.DateOffset(**wf_offset)
    return periods
    
periods= get_periods(
    start_is = "2016-01-01", 
    is_duration={'months':3}, 
    oos_duration={'months':1},
    wf_offset={'months':3},
    wf_cycles=13
)

periods

In [ ]:


best_oos_params = []
for oos_is in periods:
    start_date, end_date = oos_is['is']
    run_num = oos_is['run']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        TestStrategy,
        ema_long_period=range(20, 40),
        ema_short_period = range(3,20),)
    
    cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, best_profit, best_trades, best_equity = best_result_from_cerebro_opti_run(result)

    best_oos_params.append({'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_trades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     })
    
    print('')

print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

In [ ]:
trades_list = np.asarray(best_oos_params[4]['train_tradeslist'])
cumsum = trades_list.cumsum()
plt.plot(cumsum)

# Out-of-sample Parameters

In [ ]:
run_num = 0
for best in best_oos_params:
    start_date, end_date = best['test_period']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy, ema_long_period=best['train_param'][0], 
                                    ema_short_period = best['train_param'][1],
                                   )

    
    cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    r = result[0].analyzers.rsiA.get_analysis()
    best_oos_params[run_num]['test_numtrades'] = r['num_trades']
    best_oos_params[run_num]['test_tradeslist'] = r['trades']
    best_oos_params[run_num]['test_profit'] = r['profit']
    run_num += 1

In [ ]:
print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

# Walkforward

In [ ]:
def days_from_date_tuple(mytuple):
    #insample_days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    #insample_days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    insample_days_start = mytuple[0]
    insample_days_end = mytuple[1]
    insample_days = (insample_days_end - insample_days_start).days
    return insample_days

oos_trades = []
wfe_list = []
for values in best_oos_params:
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    insample_days = days_from_date_tuple(values['train_period'])
    oos_days = days_from_date_tuple(values['test_period'])

    insample_annual_profit = 365/insample_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / insample_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(run_num, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(insample_days, oos_days,
                                                                            insample_annual_profit, oos_annual_profit))

print('Average WFE: {}%'.format(np.asarray(wfe_list).mean()))

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):

    params = (
        ('ema_long_period', 24),
        ('ema_short_period',8),
        ('printlog', False),
        ('live', False),
        ('walkforward', None),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add Exponential MovingAverageSimple indicator
        self.ema_long = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_long_period) #kevin technical indicator chosen 
        self.ema_short = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_short_period)

        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        
        self.current_row = None
        self.wfsma = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfsma.append({
                    "ema": bt.indicators.MovingAverageExponential(self.datas[0], period=run['train_param'][0]),
                    
                    'test_period': run['test_period'],
                    'train_param': run['train_param'],
                }) 

        print(pformat(self.wfsma))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Walk-forward logic
        for row in self.wfsma:
            start, end = row['test_period']
            #period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            #period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            period_start = start
            period_end = end
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Buy    
            if self.ema_long[0] - self.ema_short[0] > 0:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    

        else:
            # Sell
            if self.ema_long[0] - self.ema_short[0] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()   
                
    def stop(self):
        self.log('(EMA long Period %2d, EMA short Period %2d) Ending Value %.2f' %
                 (self.params.ema_long_period, self.params.ema_short_period,  self.broker.getvalue()), doprint=True)


In [ ]:

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    #rsiPeriod=21, 
                    walkforward=best_oos_params,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join(r'/content/gdrive/My Drive/DecisiveWorkflowResearch/data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = best_oos_params[0]['test_period'][0]
end_date = best_oos_params[-1]['test_period'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.rsiA.get_analysis()
cumsum = np.asarray(r['trades']).cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')
avg_wfe = np.round(np.asarray(wfe_list).mean(),1)
print('Average WFE: {}%'.format(avg_wfe))

In [ ]:
# Assume CL futures margin
margin = 5000

start_date = datetime.date(2016, 1, 1)
end_date = datetime.date(2018, 1, 1)

In [ ]:
# We're going to generate a random list of trades,
# and you should replace this with your walkforward list of trades
trades_list = random.sample(range(-2000, 2300), 100)
cumsum = np.asarray(trades_list).cumsum()
plt.plot(cumsum)

In [ ]:
import random
import statistics
import logging
logger = logging.getLogger(__name__)

# MonkeyPatch Python 3.6 choices into random 3.5.5
import bisect as _bisect
import itertools as _itertools
def choices(population, weights=None, cum_weights=None, k=1):
    """Return a k sized list of population elements chosen with
replacement.
    If the relative weights or cumulative weights are not specified,
    the selections are made with equal probability.
    """
    if cum_weights is None:
        if weights is None:
            total = len(population)
            return [population[int(random.random() * total)] for i in range(k)]
        cum_weights = list(_itertools.accumulate(weights))
    elif weights is not None:
        raise TypeError('Cannot specify both weights and cumulative weights')
    if len(cum_weights) != len(population):
        raise ValueError('The number of weights does not match the population')
    bisect = _bisect.bisect
    total = cum_weights[-1]
    hi = len(cum_weights) - 1
    return [population[bisect(cum_weights, random() * total, 0, hi)] for i in range(k)]
random.choices = choices

class MonteCarlo(object):

    def __init__(self, trades_list):
        self.trades_list = trades_list
        self.num_trades_total = len(self.trades_list)
        self.num_trades_per_year = None
        self.ruin_equity = None
        self.runs = None

        self._MONTECARLO_RUNS = 2500
        logger.info("Initialize \t| Trades: {} \t| MC Runs: {}".format(self.num_trades_total, self._MONTECARLO_RUNS))

    def settings(self, ruin_equity, start_date, end_date):
        self._set_ruin_equity(ruin_equity)
        self._set_trades_per_year(start_date, end_date)
        logger.info("Settings \t| Ruin: {} \t| Trades Per Year: {}".format(self.ruin_equity, self.num_trades_per_year))

    def _set_ruin_equity(self, ruin_equity):
        self.ruin_equity = ruin_equity

    def _set_trades_per_year(self, start_date, end_date):
        td = end_date - start_date
        self.num_trades_per_year = int(self.num_trades_total * 365 / td.days)
        logger.debug("TimeDelta: {} {} trades/yr".format(td, self.num_trades_per_year))

    def _random_trade(self, starting_equity):
        assert self.num_trades_per_year
        assert self.ruin_equity

        trades = random.choices(self.trades_list, k=self.num_trades_per_year)
        #logger.debug("{} {}".format(len(trades), trades))

        # Check for ruin at any point in the trades list
        is_ruined = 0
        equity = starting_equity
        for trade in trades:
            equity = equity + trade
            if equity < self.ruin_equity:
                is_ruined = 1
                break

        stats = {'profit': sum(trades),
                'returns_pct': int(100 * ((starting_equity + sum(trades))/starting_equity - 1)),
                'drawdown_pct': self._drawdown(starting_equity, trades),
                 'is_ruined': is_ruined,
                 'is_profitable': 1 if sum(trades) >= 0 else 0,
                }
        stats['returns_per_drawdown'] = stats['returns_pct'] / stats['drawdown_pct']
        logger.debug(stats)
        return stats

    def _drawdown(self, starting_equity, trades):
        """Returns the maximum drawdown in a set of trades"""
        equity = starting_equity
        hwm = starting_equity
        max_drawdown_pct = 0
        drawdown_pct = 0

        for trade in trades:
            equity = equity + trade

            # Set High Water Mark
            if equity > hwm:
                hwm = equity

            # Look for drawdown
            if equity < hwm:
                drawdown_pct = (100*(1 - (equity / hwm)))
                if drawdown_pct > max_drawdown_pct:
                    max_drawdown_pct = drawdown_pct

            #logger.debug("{} eq:{} \t hwm:{} \t dd:{} mdd:{}".format(trade, equity, hwm, drawdown_pct, max_drawdown_pct))
        return max_drawdown_pct


    def _median_stats_run(self, starting_equity):
        montecarlo = {}
        median_montecarlo = {}

        # runs
        for _ in range(self._MONTECARLO_RUNS):

            # make the list for every key
            stats = self._random_trade(starting_equity)
            for k, v in stats.items():
                if k not in montecarlo.keys():
                    montecarlo[k] = []
                montecarlo[k].append(v)

        # run statistics on all the lists of every key
        for k, v in montecarlo.items():
            # ignore non-median stats
            if k != "is_ruined" or k != "is_profitable":
                median_montecarlo[k] = statistics.median(montecarlo[k])

        logger.debug((montecarlo['profit']))
        logger.debug((montecarlo['is_ruined']))
        logger.debug(sum(montecarlo['is_ruined']))
        median_montecarlo['is_ruined'] = 100 * sum(montecarlo['is_ruined']) / self._MONTECARLO_RUNS
        median_montecarlo['is_profitable'] = 100 * sum(montecarlo['is_profitable']) / self._MONTECARLO_RUNS
        median_montecarlo['equity'] = starting_equity

        # calculate risk of ruin
        logger.debug("Median {}: {}".format(starting_equity, median_montecarlo))

        return median_montecarlo

    def run(self, base_equity, steps=11):
        step_size = int(base_equity/4)
        end_eq = base_equity + step_size * steps
        starting_equities_list = range(base_equity, end_eq, step_size)
        return self._run_equity_list(starting_equities_list)

    def _run_equity_list(self, starting_equities_list):
        runs = []
        for starting_equity in starting_equities_list:
            runs.append(self._median_stats_run(starting_equity))
        self.runs = runs
        return runs

    def best_run(self, target_risk_of_ruin_pct=10):
        assert self.runs
        for run in self.runs:
            if run['is_ruined'] < target_risk_of_ruin_pct:
                return run
        return None

In [ ]:
# Initialize
mc = MonteCarlo(trades_list)

# We will sample with replacement the number of trades per year
# so we need the start and end date to determine how many trades at in a year on average
mc.settings(margin, start_date, end_date)

# Test different levels of equity starting at this value
trial_starting_equity = int(margin * 1.5)

# Run the Monte Carlo
results = mc.run(trial_starting_equity)

In [ ]:
# Put the results in a dataframe so it's nicer to look at in notebook
# Our goal is to get the highest equity below 10% Risk of Ruin
df = pd.DataFrame(index=range(1,len(results)))
count = 1
for result in results:
    df.loc[count, 'equity'] = result['equity']
    df.loc[count, 'is_ruined'] = result['is_ruined']
    df.loc[count, 'is_profitable'] = result['is_profitable']
    df.loc[count, 'returns_pct'] = result['returns_pct']
    df.loc[count, 'drawdown_pct'] = result['drawdown_pct']
    df.loc[count, 'returns_per_drawdown'] = result['returns_per_drawdown']
    count += 1

# Get the recommended values
recommended = df[df['is_ruined'] <= 10].iloc[0]
print('Recommend a starting equity of {}, which has {:0.2}% Risk-of-Ruin, \n\t{:0.0f}% Probability-of-Profit and a {:0.2f} Returns/Drawdown Ratio'.format(
                recommended['equity'], recommended['is_ruined'], 
                recommended['is_profitable'], recommended['returns_per_drawdown']))

if recommended['is_ruined'] > 10 or recommended['returns_per_drawdown'] < 2.0:
    print("Risk Assessment: FAILED")
else:
    print("Risk Assessment: PASSED")

mc_1p5x = recommended['drawdown_pct'] * 1.5
print("MC-Drawdown: {:0.1f}% MC-1.5x-DD: {:0.1f}%".format(recommended['drawdown_pct'], mc_1p5x))

profit = recommended['equity'] * recommended['returns_pct'] / 100
months = (end_date - start_date).days/30
average_monthly_net_profit = profit / months
print("Average monthly net profit: {:0.1f}".format(average_monthly_net_profit))

df

In [ ]:
plt.plot(cumsum + recommended['equity'])